In [1]:
using Pkg, Revise
Pkg.activate(".")

  Activating environment at `~/Desktop/SelectivelyAdaptiveLasso/Project.toml`


In [2]:
import SelectivelyAdaptiveLasso as SAL

┌ Info: Precompiling SelectivelyAdaptiveLasso [52f33b2b-83b2-4cfe-8aa4-4763af86ef4f]
└ @ Base loading.jl:1342
┌ Warning: Package SelectivelyAdaptiveLasso does not have Combinatorics in its dependencies:
│ - If you have SelectivelyAdaptiveLasso checked out for development and have
│   added Combinatorics as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with SelectivelyAdaptiveLasso
└ Loading Combinatorics into SelectivelyAdaptiveLasso from project dependency, future warnings for SelectivelyAdaptiveLasso are suppressed.


In [3]:
import Statistics: mean
import Distributions as Dist
import LinearAlgebra: diagm

n, p = 5000, 50

X_dist = Dist.reshape(
    Dist.MvNormal(zeros(p), 0.1ones(p,p) + 0.9diagm(ones(p))), 
    (1,p)
)

X = vcat(rand(X_dist, n)...);
# Y = rand(n)
Y = 5X[:,1] + X[:,2].^2 - 3X[:,3].*X[:,4]
Y = Y .- mean(Y)

n_val = Int(round(n/2))
X_val, Y_val = X[1:n_val,:], Y[1:n_val]
X, Y = X[(n_val+1):end,:], Y[(n_val+1):end];

In [7]:
sal_spec = SAL.SALSpec(
    max_iter=50,
    m_subsample=200,
    n_subsample=200
)
sal_fit, log = SAL.fit(
    sal_spec, 
    X, Y, 
    X_val=X_val, Y_val=Y_val
)
loss, loss_val, _, _ = log;

  0.046114 seconds (71 allocations: 47.341 MiB)
  0.175027 seconds (10.06 k allocations: 5.352 MiB)
  0.012891 seconds (31 allocations: 80.516 KiB)
  0.000011 seconds (4 allocations: 39.281 KiB)
(36.965985843220245, 36.391149441891336, 1.0, Set([2, 1]))
  0.111105 seconds (71 allocations: 47.341 MiB, 49.16% gc time)
  0.012375 seconds (31 allocations: 80.516 KiB)
  0.000010 seconds (4 allocations: 39.281 KiB)
(36.965985843220245, 36.391149441891336, 0.6666666666666666, Set([2, 1]))
  0.041199 seconds (54 allocations: 31.832 MiB)
  0.208356 seconds (10.04 k allocations: 5.218 MiB)
  0.023636 seconds (31 allocations: 80.516 KiB)
  0.000009 seconds (4 allocations: 39.281 KiB)
(36.965985843220245, 36.391149441891336, 0.6666666666666666, Set([3]))
  0.031116 seconds (54 allocations: 31.832 MiB)
  0.134169 seconds (10.02 k allocations: 4.296 MiB)
  0.034202 seconds (31 allocations: 80.516 KiB)
  0.000010 seconds (4 allocations: 39.281 KiB)
(36.965985843220245, 36.391149441891336, 0.666666666